In [ ]:
#importing libraries
import pandas as pd
import numpy as np
import seaborn as sns
import string
import nltk



In [ ]:
df=pd.read_csv("tweets.csv")

In [ ]:
df.head()

,id,label,tweet
0,1,0,#fingerprint #Pregnancy Test https://goo.gl/h1...
1,2,0,Finally a transparant silicon case ^^ Thanks t...
2,3,0,We love this! Would you go? #talk #makememorie...
3,4,0,I'm wired I know I'm George I was made that wa...
4,5,1,What amazing service! Apple won't even talk to...


In [ ]:
data=df[['tweet','label']]

In [ ]:
data['label'].unique()

array([0, 1])

In [ ]:
data_pos = data[data['label'] == 1]
data_neg = data[data['label'] == 0]

In [ ]:
data_pos = data_pos.iloc[:int(20000)]
data_neg = data_neg.iloc[:int(20000)]

In [ ]:
dataset = pd.concat([data_pos, data_neg])

In [ ]:
dataset['tweet']=dataset['tweet'].str.lower()
dataset['tweet'].tail()

7915    live out loud #lol #liveoutloud #selfie #smile...
7916    we would like to wish you an amazing day! make...
7917    helping my lovely 90 year old neighbor with he...
7918    finally got my #smart #pocket #wifi stay conne...
7919    apple barcelona!!! #apple #store #bcn #barcelo...
Name: tweet, dtype: object

In [ ]:
import string
def remove_punctuation(tweet):
  punctuationfree="".join([i for i in tweet if i not in string.punctuation])
  return punctuationfree
  #to remove all punctuations

In [ ]:
import nltk
nltk.download('punkt')
def tokenization(tweet):
  words=nltk.word_tokenize(tweet)
  return words

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
nltk.download('stopwords')
stopwords=nltk.corpus.stopwords.words('english')
def remove_stopwords(tweet):
  output=[i for i in tweet if i not in stopwords]
  return output

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
import re
def cleaning_repeating_char(tweet):
    return re.sub(r'(.)1+', r'1', tweet)
dataset['tweet'] = dataset['tweet'].apply(lambda x: cleaning_repeating_char(x))
dataset['tweet'].tail()

7915    live out loud #lol #liveoutloud #selfie #smile...
7916    we would like to wish you an amazing day! make...
7917    helping my lovely 90 year old neighbor with he...
7918    finally got my #smart #pocket #wifi stay conne...
7919    apple barcelona!!! #apple #store #bcn #barcelo...
Name: tweet, dtype: object

In [ ]:
def cleaning_URLs(data):
    return re.sub('((www.[^s]+)|(https?://[^s]+))',' ',data)
dataset['tweet'] = dataset['tweet'].apply(lambda x: cleaning_URLs(x))
dataset['tweet'].tail()

7915    live out loud #lol #liveoutloud #selfie #smile...
7916    we would like to wish you an amazing day! make...
7917    helping my lovely 90 year old neighbor with he...
7918    finally got my #smart #pocket #wifi stay conne...
7919    apple barcelona!!! #apple #store #bcn #barcelo...
Name: tweet, dtype: object

In [ ]:
def cleaning_numbers(data):
    return re.sub('[0-9]+', '', data)
dataset['tweet'] = dataset['tweet'].apply(lambda x: cleaning_numbers(x))
dataset['tweet'].tail()

7915    live out loud #lol #liveoutloud #selfie #smile...
7916    we would like to wish you an amazing day! make...
7917    helping my lovely  year old neighbor with her ...
7918    finally got my #smart #pocket #wifi stay conne...
7919    apple barcelona!!! #apple #store #bcn #barcelo...
Name: tweet, dtype: object

In [ ]:
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'w+')
dataset['tweet'] = dataset['tweet'].apply(tokenizer.tokenize)
dataset['tweet'].head()

4        [w, w]
5           [w]
10    [w, w, w]
11       [w, w]
12    [w, w, w]
Name: tweet, dtype: object

In [ ]:
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
wordnet_lemmatizer=WordNetLemmatizer()
def lemmatizer(tweet):
  lemm_text=[wordnet_lemmatizer.lemmatize(word) for word in tweet]
  return lemm_text

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
import nltk
st = nltk.PorterStemmer()
def stemming_on_text(data):
    text = [st.stem(word) for word in data]
    return data
dataset['tweet']= dataset['tweet'].apply(lambda x: stemming_on_text(x))
dataset['tweet'].head()

4        [w, w]
5           [w]
10    [w, w, w]
11       [w, w]
12    [w, w, w]
Name: tweet, dtype: object

In [ ]:
lm = nltk.WordNetLemmatizer()
def lemmatizer_on_text(data):
    tweet = [lm.lemmatize(word) for word in data]
    return data
dataset['tweet'] = dataset['tweet'].apply(lambda x: lemmatizer_on_text(x))
dataset['tweet'].head()

4        [w, w]
5           [w]
10    [w, w, w]
11       [w, w]
12    [w, w, w]
Name: tweet, dtype: object

In [ ]:
X=data.tweet
y=data.label

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.05, random_state =26105111)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectoriser = TfidfVectorizer(ngram_range=(1,2), max_features=500000)
vectoriser.fit(X_train)


TfidfVectorizer(max_features=500000, ngram_range=(1, 2))

In [ ]:
X_train = vectoriser.transform(X_train)
X_test  = vectoriser.transform(X_test)

In [ ]:
def preprocess(df_col):
  corpus=[]
  for item in df_col:
    new_item=remove_punctuation(item)
    new_item=new_item.lower()
    new_item=tokenization(new_item)
    new_item=remove_stopwords(new_item)
    new_item=lemmatizer(new_item)
    corpus.append(''.join(str(x)for x in new_item))
  return corpus

In [ ]:
X.dtype

dtype('int64')

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report
log_reg=LogisticRegression()
log_reg.fit(X_train,y_train)

y_pred=log_reg.predict(X_test)



In [ ]:
from sklearn import metrics
metrics.accuracy_score(y_pred,y)

ValueError: Found input variables with inconsistent numbers of samples: [396, 7920]